<p align="center">
  <img src="imagenes/3.png">
</p>
En el Repo vamos a conectarnos a la api de rick and morty para obtener datos de la serie, mas precisamente los <span style="color: red; font-weight: bold;">Personajes</span>, <span style="color: blue; font-weight: bold;">Capítulos</span> y <span style="color: green; font-weight: bold;">Locaciones</span>, y lo vamos a subir a redshift.


In [148]:
# Librerias que vamos a utlizar
import requests
import psycopg2
import pandas as pd
from psycopg2.extras import execute_values

<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/9.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: red; font-size: 30px; text-align: center;"><b>Nos conectamos a la API</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>


Creamos una funcion que no permite traer todos los datos de cada uno de  los <span style="color: red; font-weight: bold;">Personajes</span>, <span style="color: blue; font-weight: bold;">Capítulos</span> y <span style="color: green; font-weight: bold;">Locaciones</span>,cada vez que lo requiramos.

In [2]:
def traer_info(dato):
    url = f"https://rickandmortyapi.com/api/{dato}"
    datos_obtenidos = []

    while url is not None:
        response = requests.get(url)
        data = response.json()
        datos_obtenidos += data['results']
        url = data['info']['next']
    # Lo convertimos en un dataframe
    df = pd.DataFrame(datos_obtenidos)
    return df

<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/13.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: red; font-size: 30px; text-align: center;"><b>Trabajamos con los datos de los personajes</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>

In [143]:
df_personajes  =  traer_info('character')
df_personajes.head(4) # vemos que tenemos en el dataframe

,id,name,status,species,type,gender,origin,location,image,episode,url,created
0,1,Rick Sanchez,Alive,Human,,Male,"{'name': 'Earth (C-137)', 'url': 'https://rick...","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,"{'name': 'unknown', 'url': ''}","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z
3,4,Beth Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z


In [144]:
df_personajes["origen_name"] = df_personajes["origin"].apply(lambda x: x["name"]) # creamos una columna conde solo salga el nombre de origen
df_personajes = df_personajes[["id","name" ,"status", "species", "type", "gender", "origen_name", "created"]] # creamos el dataframe con la info que queremos 
df_personajes.columns = ["id", "nombre", "estado", "especie", "tipo", "genero", "origen", "fecha_creacion"] # Cambiamos los nombres de las columnas
df_personajes.head(2) # Mostramos como quedo

,id,nombre,estado,especie,tipo,genero,origen,fecha_creacion
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,unknown,2017-11-04T18:50:21.651Z


In [145]:
#sacamos los espacios en blanco de la columna tipo
df_personajes.loc[:,"tipo"] =df_personajes["tipo"].replace('','no aplica')
df_personajes.head(4) # Data final

,id,nombre,estado,especie,tipo,genero,origen,fecha_creacion
0,1,Rick Sanchez,Alive,Human,no aplica,Male,Earth (C-137),2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,no aplica,Male,unknown,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,no aplica,Female,Earth (Replacement Dimension),2017-11-04T19:09:56.428Z
3,4,Beth Smith,Alive,Human,no aplica,Female,Earth (Replacement Dimension),2017-11-04T19:22:43.665Z


<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/13.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: green; font-size: 30px; text-align: center;"><b>Trabajamos con los datos de las locaciones</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>

In [98]:
df_locaciones  =  traer_info('location')
df_locaciones.head(4)#vemos que datos tenemos

,id,name,type,dimension,residents,url,created
0,1,Earth (C-137),Planet,Dimension C-137,"[https://rickandmortyapi.com/api/character/38,...",https://rickandmortyapi.com/api/location/1,2017-11-10T12:42:04.162Z
1,2,Abadango,Cluster,unknown,[https://rickandmortyapi.com/api/character/6],https://rickandmortyapi.com/api/location/2,2017-11-10T13:06:38.182Z
2,3,Citadel of Ricks,Space station,unknown,"[https://rickandmortyapi.com/api/character/8, ...",https://rickandmortyapi.com/api/location/3,2017-11-10T13:08:13.191Z
3,4,Worldender's lair,Planet,unknown,"[https://rickandmortyapi.com/api/character/10,...",https://rickandmortyapi.com/api/location/4,2017-11-10T13:08:20.569Z


In [104]:
df_locaciones = df_locaciones.drop(columns=['residents','url']) # Borramos las columnas que no necesitamos

In [106]:
df_locaciones.columns = ["id", "nombre", "tipo", "dimension", "fecha_creacion"] # Cambiamos los nombres de las columnas
df_locaciones.head(4) # Data final

,id,nombre,tipo,dimension,fecha_creacion
0,1,Earth (C-137),Planet,Dimension C-137,2017-11-10T12:42:04.162Z
1,2,Abadango,Cluster,unknown,2017-11-10T13:06:38.182Z
2,3,Citadel of Ricks,Space station,unknown,2017-11-10T13:08:13.191Z
3,4,Worldender's lair,Planet,unknown,2017-11-10T13:08:20.569Z


<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/13.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: blue; font-size: 30px; text-align: center;"><b>Trabajamos con los datos de los capitulos</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>

In [108]:
df_episodios  =  traer_info('episode')
df_episodios.head(4)

,id,name,air_date,episode,characters,url,created
0,1,Pilot,"December 2, 2013",S01E01,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/1,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/2,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/3,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/4,2017-11-10T12:56:34.129Z


In [110]:
df_episodios = df_episodios.drop(columns=['characters','url']) # Borramos las columnas que no necesitamos

In [114]:
df_episodios.columns = ["id","nombre_episodio", "fecha_aire", "episodio","fecha_creacion"] # Cambiamos los nombres de las columnas
df_episodios.head(4) # Data final

,id,nombre_episodio,fecha_aire,episodio,fecha_creacion
0,1,Pilot,"December 2, 2013",S01E01,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,2017-11-10T12:56:34.129Z


<p align="center">
  <img src="imagenes/5.png" width="40%">
  <br>
  <span style="color: red;"><b>Nos conectamos a Redshift</b></span>
</p>

In [171]:
with open("C:/Users/jorge/Desktop/Nueva carpeta/api/pass.txt", "r") as archivo:
    password = archivo.read()

In [172]:
conn = psycopg2.connect(
    host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
    port = '5439',
    dbname = 'data-engineer-database',
    user = 'jorgeflores2311233_coderhouse',
    password = password
)
if conn.status == psycopg2.extensions.STATUS_READY:
    print("0K -- CONECTADO A LA BASE DE DATOS --")

0K -- CONECTADO A LA BASE DE DATOS --


<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/13.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: red; font-size: 30px; text-align: center;"><b>Creamos la tabla para personajes  y subimos los datos</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>


In [173]:
with conn.cursor() as cur:
    cur.execute("""
       DROP TABLE IF  EXISTS jorgeflores2311233_coderhouse.personaje;
     """)
    cur.execute("""
        CREATE TABLE jorgeflores2311233_coderhouse.personaje(
        id INTEGER PRIMARY KEY,
        nombre VARCHAR(250),
        estado VARCHAR(250),
        especie VARCHAR(250),
        tipo VARCHAR(250),
        genero VARCHAR(250),
        origen  VARCHAR(250),
        fecha_creacion  DATETIME       
        );
      """)
    tabla = "personaje"
    columns = ['id', 'nombre', 'estado', 'especie', 'tipo', 'genero', 'origen','fecha_creacion']
    values = [tuple(x) for x in df_personajes.to_numpy()]
    insert_sql = f"INSERT INTO {tabla} ({', '.join(columns)}) VALUES %s"
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    conn.commit()
    print("2 -- TABLA CREADA CON EXITO --")

2 -- TABLA CREADA CON EXITO --


<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/13.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: red; font-size: 30px; text-align: center;"><b>Creamos la tabla para episodios  y subimos los datos</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>


In [174]:
df_episodios.head(4)

,id,nombre_episodio,fecha_aire,episodio,fecha_creacion
0,1,Pilot,"December 2, 2013",S01E01,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,2017-11-10T12:56:34.129Z


In [175]:
with conn.cursor() as cur:
    cur.execute("""
       DROP TABLE IF  EXISTS jorgeflores2311233_coderhouse.episodio;
     """)
    cur.execute("""
        CREATE TABLE jorgeflores2311233_coderhouse.episodio(
        id INTEGER PRIMARY KEY,
        nombre_episodio VARCHAR(250),
        fecha_aire VARCHAR(250),
        episodio VARCHAR(250),
        fecha_creacion DATETIME       
        );
      """)
    tabla = "episodio"
    columns = ['id', 'nombre_episodio', 'fecha_aire', 'episodio', 'fecha_creacion']
    values = [tuple(x) for x in df_episodios.to_numpy()]
    insert_sql = f"INSERT INTO {tabla} ({', '.join(columns)}) VALUES %s"
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    conn.commit()
    print("2 -- TABLA CREADA CON EXITO --")

2 -- TABLA CREADA CON EXITO --


<p style="display: flex; align-items: center; justify-content: center; width: 100%;">
  <img src="imagenes/13.png" width="10%" style="float: left; margin-right: 30px;">
  <span style="color: red; font-size: 30px; text-align: center;"><b>Creamos la tabla para locaciones  y subimos los datos</b></span>
  <img src="imagenes/8.png" width="10%" style="float: right; margin-left: 10px;">
</p>


In [176]:
with conn.cursor() as cur:
    cur.execute("""
       DROP TABLE IF  EXISTS jorgeflores2311233_coderhouse.locacion;
     """)
    cur.execute("""
        CREATE TABLE jorgeflores2311233_coderhouse.locacion(
        id INTEGER PRIMARY KEY,
        nombre_episodio VARCHAR(250),
        fecha_aire VARCHAR(250),
        episodio VARCHAR(250),
        fecha_creacion DATETIME       
        );
      """)
    tabla = "locacion"
    columns = ['id', 'nombre_episodio', 'fecha_aire', 'episodio', 'fecha_creacion']
    values = [tuple(x) for x in df_locaciones.to_numpy()]
    insert_sql = f"INSERT INTO {tabla} ({', '.join(columns)}) VALUES %s"
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    conn.commit()
    print("2 -- TABLA CREADA CON EXITO --")

2 -- TABLA CREADA CON EXITO --


cerrramos conexion con la base de datos

In [177]:
conn.close()
print(f"ok -- CONEXION CERRADA -- \n{'='*25}")

ok -- CONEXION CERRADA -- 
